Please ensure you have run all previous notebooks in sequence before running this. This notebook uses image from ACI notebook for deploying to AKS.

In this lab we are going to deploy our image to AKS instead of ACI.  

We could certainly use an existing AKS cluster with AMLS, the syntax below would be a bit different.  Since we don't have an AKS service in our Azure subscription yet, it's easier to just use AMLS to deploy AKS resources.  

In many cases, frankly, it may make a lot more sense to create many (or a few) AKS clusters for your ML workloads.  You may want to have an expensive GPU-based cluster for Deep Learning training and another cluster that is much smaller/cheaper for inferencing.

Notice the scaffolding code...

In [4]:
from azureml.core import Workspace
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

#'''
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')
#'''

In [5]:
# List images by ws

from azureml.core.image import ContainerImage
for i in ContainerImage.list(workspace = ws):
    print('{}(v.{} [{}]) stored at {} with build log {}'.format(i.name, i.version, i.creation_state, i.image_location, i.image_build_log_uri))

In [6]:
from azureml.core.image import Image
myimage = Image(workspace=ws, id="bikebuyeraciws:1")

The next cell builds the AKS cluster.  Look in the portal at the options available to create an AKS cluster from within AMLS.  Note that we have not specfied every option.  There are defaults.  We can also watch the provisioning status in the Azure portal.  Note that you may see SUCCESS in the portal _before_ you see it below.

In [8]:
#create AKS compute
#it may take 20-25 minutes to create a new cluster

from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'amls-aks-clus1' 

# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

aks_target.wait_for_completion(show_output = True)

print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

In [9]:
from azureml.core.webservice import Webservice
help( Webservice.deploy_from_image)

As we run the next cell we should see a second deployment in the portal.

In [11]:
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import ContainerImage

#Set the web service configuration (using default here)
aks_config = AksWebservice.deploy_configuration()

#unique service name
service_name ='bb-aks-service'

# Webservice creation using single command, there is a variant to use image directly as well.
aks_service = Webservice.deploy_from_image(
  workspace=ws, 
  name=service_name,
  deployment_config = aks_config,
  image = myimage,
  deployment_target = aks_target
    )

aks_service.wait_for_deployment(show_output=True)

We can get all of the following information from the portal, but let's do it programmatically.

In [13]:
#for using the Web HTTP API 
print(aks_service.scoring_uri)
print(aks_service.get_keys())

In [14]:
import json

#get the some sample data
test_data_path = "BikeBuyerTest"
test = spark.read.parquet(test_data_path).limit(5)

test_json = json.dumps(test.toJSON().collect())

print(test_json)

In [15]:
#using data defined above predict if customer will buy a bike Yes (1) or No (0)
aks_service.run(input_data=test_json)

In [16]:
#comment to not delete the web service
#aks_service.delete()
#image.delete()
#model.delete()
#aks_target.delete()